In [ ]:
import numpy as np
import xgboost as xgb

# 自定义 objective function
def custom_obj(preds, dtrain):
    # 读取标签（0 表示 No Risk, 1 表示 Risk）
    labels = dtrain.get_label()  # shape: (n_samples,)
    # 读取贷款金额信息（确保在构造 DMatrix 时设置了此信息）
    loan_amount = dtrain.get_float_info('LoanAmount')
    
    # 计算预测概率（XGBoost 传入的是 raw score，即 logit）
    p = 1.0 / (1.0 + np.exp(-preds))
    
    # 定义权重：按照实际比例和训练比例计算
    weight_risk = 0.02 / (1/3)    # 对 Risk 样本
    weight_no_risk = 0.98 / (2/3)   # 对 No Risk 样本
    
    # 针对每个样本确定 A, B 和对应权重
    # 对于标签为 1 (Risk): A = 1.0, B = 5.0 + 0.6 * LoanAmount, 权重为 weight_risk
    # 对于标签为 0 (No Risk): A = 1.0, B = 1.0 - 0.05 * LoanAmount, 权重为 weight_no_risk
    A = np.ones_like(labels)
    B = np.where(labels == 1, 5.0 + 0.6 * loan_amount, 1.0 - 0.05 * loan_amount)
    custom_weight = np.where(labels == 1, weight_risk, weight_no_risk)
    
    # 计算梯度和 Hessian
    # 损失 L = custom_weight * [B + (A-B) * p]
    # 梯度 dL/df = custom_weight * (A-B) * p * (1-p)
    grad = custom_weight * (A - B) * p * (1 - p)
    # Hessian d^2L/df^2 = custom_weight * (A-B) * p*(1-p)*(1-2p)
    hess = custom_weight * (A - B) * p * (1 - p) * (1 - 2*p)
    
    return grad, hess

# 使用示例：
# 假设 X_train, y_train, loan_amount_train 已经准备好，其中 y_train 为 0/1
dtrain = xgb.DMatrix(X_train, label=y_train)
# 将 loan_amount 作为 float 信息传入
dtrain.set_float_info('LoanAmount', loan_amount_train)

# 定义参数（其他参数自行设定）
params = {
    'max_depth': 4,
    'eta': 0.1,
    'silent': 1,
    # 注意这里 objective 可设为 'binary:logistic'，但当指定自定义 objective 时此项不影响训练
}

# 训练时传入自定义 objective
bst = xgb.train(params, dtrain, num_boost_round=100, obj=custom_obj)


TypeError: bad operand type for unary -: 'DMatrix'